In [1]:
import getfem as gf
import numpy as np

In [2]:
E = 1000.0  # N/mm2
A = 1.0  # mm2
l = 1.0  # mm
P = 20.0  # N
g = 0.01  # mm
elements_degree = 1
p = 1000

In [3]:
mesh1 = gf.Mesh("cartesian", np.array([0.0, 1.0 * l, 2.0 * l, 3.0 * l]))
mesh2 = gf.Mesh("cartesian", np.array([[3.0 * l, 3.0 * l + l]]))
# mesh2 = gf.Mesh("cartesian", np.array([[3.0 * l + g, 3.0 * l + g + l]]))

In [4]:
fb1 = mesh1.outer_faces_with_direction([-1.0], 0.01)
fb2 = mesh2.outer_faces_with_direction([-1.0], 0.01)
fb3 = mesh1.outer_faces_with_direction([1.0], 0.01)
fb4 = mesh2.outer_faces_with_direction([1.0], 0.01)

LEFT_BOUND = 1
RIGHT_BOUND = 2

mesh1.set_region(LEFT_BOUND, fb1)
mesh2.set_region(LEFT_BOUND, fb2)
mesh1.set_region(RIGHT_BOUND, fb3)
mesh2.set_region(RIGHT_BOUND, fb4)

In [5]:
mfu1 = gf.MeshFem(mesh1, 1)
mfu2 = gf.MeshFem(mesh2, 1)
mfu1.set_classical_fem(elements_degree)
mfu2.set_classical_fem(elements_degree)
mim1 = gf.MeshIm(mesh1, pow(elements_degree, 2))
mim2 = gf.MeshIm(mesh2, pow(elements_degree, 2))

In [6]:
md = gf.Model("real")
md.add_fem_variable("u1", mfu1) # Displacement of the structure 1
md.add_fem_variable("u2", mfu2) # Displacement of the structure 2

In [7]:
md.add_initialized_data("EA", [E*A])
md.add_nonlinear_term(mim1, "EA*Grad_u1.Grad_Test_u1")
md.add_nonlinear_term(mim2, "2.0*EA*Grad_u2.Grad_Test_u2")

F = np.array([0.0, 0.0, 1.0, 0.0])
md.add_explicit_rhs("u1", F)

2

In [8]:
md.add_initialized_data("r1", [0.0])
md.add_initialized_data("H1", [[1.0]])
md.add_initialized_data("r2", [0.0])
md.add_initialized_data("H2", [[1.0]])
md.add_generalized_Dirichlet_condition_with_multipliers(
    mim1, "u1", mfu1, LEFT_BOUND, "r1", "H1"
)
md.add_generalized_Dirichlet_condition_with_multipliers(
    mim2, "u2", mfu2, RIGHT_BOUND, "r2", "H2"
)

4

In [9]:
# md.add_initialized_data("n", [1.0])
md.add_initialized_data("r", [p])
# BN1 = gf.Spmat("empty",2,2)
# BN1[0, 0] = 1.0
# BN1[0, 1] = -1.0
# BN1[1, 0] = -1.0
# BN1[1, 1] = 1.0
# BN2 = gf.Spmat("empty",2,2)
# BN2[0, 0] = 1.0
# BN2[0, 1] = -1.0
# BN2[1, 0] = -1.0
# BN2[1, 1] = 1.0
# md.add_basic_contact_brick_two_deformable_bodies("u1", "u2", "n", "r", BN1, BN2)
md.add_penalized_contact_between_nonmatching_meshes_brick(mim1, "u1", "u2", "r", RIGHT_BOUND, LEFT_BOUND)

5

In [10]:
md.assembly("build_all")
SM = md.tangent_matrix()
print(SM.full())

[[ 0.e+00  0.e+00  1.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  1.e+00]
 [ 1.e+00  0.e+00  1.e+03 -1.e+03  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  0.e+00 -1.e+03  2.e+03 -1.e+03  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00 -1.e+03  2.e+03 -1.e+03  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00 -1.e+03  2.e+03 -1.e+03  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -1.e+03  3.e+03 -2.e+03]
 [ 0.e+00  1.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -2.e+03  2.e+03]]


In [11]:
rhs = md.rhs()
print(rhs)

[0. 0. 0. 0. 1. 0. 0. 0.]


In [12]:
md.solve()

(1, 1)

In [13]:
U1 = md.variable("u1")
print(U1)

[0.         0.00055556 0.00111111 0.00066667]


In [14]:
U2 = md.variable("u2")
print(U2)

[ 2.22222222e-04 -2.98744089e-20]
